# **Performing Pose Estimation using AlphaPose**

Clone repository:

In [1]:
!git clone https://github.com/christinathr2000/TennisCoach.git

Cloning into 'TennisCoach'...
remote: Enumerating objects: 291, done.
remote: Counting objects: 100% (291/291), done.
remote: Compressing objects: 100% (241/241), done.
remote: Total 291 (delta 40), reused 243 (delta 27), pack-reused 0
Receiving objects: 100% (291/291), 52.31 MiB | 26.13 MiB/s, done.
Resolving deltas: 100% (40/40), done.


Install requirements:

In [2]:
cd /content/TennisCoach/AlphaPose

/content/TennisCoach/AlphaPose


In [18]:
!pip install -q -r requirements.txt

Checks if GPU is available:

In [4]:
import torch
print(torch.cuda.is_available())

True


**Downloads used SPPE and YOLO detector and stores in correct folder:** \\
Lately, an access denied error occurs when using gdown, even though this should not happen.
If you get that message, download the file at the shown link and add to
/TennisCoach/AlphaPose/models/sppe folder.\\
Below, you'll find a code snippet that served as our solution for the gdown error. We stored all the data on our drive and imported it from there. Feel free to follow the same approach.

In [ ]:
cd models/sppe

/content/TennisCoach/AlphaPose/models/sppe


In [ ]:
!gdown --id 1OPORTWB2cwd5YTVBX-NE8fsauZJWsrtW
# Lately, a access denied error occurs - even though this should not happen.
# If you get that message, download the file at the shown link and add to /TennisCoach/AlphaPose/models/sppe folder

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1OPORTWB2cwd5YTVBX-NE8fsauZJWsrtW 



In [ ]:
cd ../yolo

/content/TennisCoach/AlphaPose/models/yolo


In [ ]:
!gdown --id 1D47msNOOiJKvPOXlnpyzdKA3k6E97NTC
# Lately, a access denied error occurs - even though this should not happen.
# If you get that message, download the file at the shown link and add to /TennisCoach/AlphaPose/models/yolo folder
# or use cells underneath to get data from Google Drive

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1D47msNOOiJKvPOXlnpyzdKA3k6E97NTC 



In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import shutil
sppe_path = '/content/drive/My Drive/PersonalTennisCoach/duc_se.pth'
new_sppe_path = '/content/TennisCoach/AlphaPose/models/sppe/'
shutil.copy(sppe_path, new_sppe_path)

yolo_path = '/content/drive/My Drive/PersonalTennisCoach/yolov3-spp.weights'
new_yolo_path = '/content/TennisCoach/AlphaPose/models/yolo/'
shutil.copy(yolo_path, new_yolo_path)

'/content/TennisCoach/AlphaPose/models/yolo/yolov3-spp.weights'

**Enter Video Number:** \\
We provide some test videos on our github and can be found in the input folder.
Feel free to set video_nr to your selected number.

In [19]:
video_nr = 11
long_nr = str('{:03d}'.format(video_nr))
video_path = 'input/video' + long_nr + '.mp4'

#Video Preprocessing
We check framerate, resolution and video format. \\
The histogram equalization part is commented out because AlphaPose also did not use it during training.


In [8]:
cd /content/TennisCoach/AlphaPose

/content/TennisCoach/AlphaPose


In [9]:
import cv2
import magic

def video_preprocessing(video_path):
    #Check if file is mp4
    file_type = magic.Magic().from_file(video_path)
    if 'MP4' not in file_type.upper():
      print("Error: The file format is not MP4.")
      return False

    video = cv2.VideoCapture(video_path)

    if not video.isOpened():
        print("Error: Video file was not opened successfully")
        return False

    # Get the frames per second (fps) of the video and check if minimum of 20 fps is met
    fps = video.get(cv2.CAP_PROP_FPS)
    if fps < 13:
      print("Error: Frame rate is too low. A minimum rate of 20 fps is required.")
      video.release()
      return False

    #Check resolution of video
    width = int(video.get(3))
    height = int(video.get(4))
    resolution = (width, height)

    if (width < 300 or height < 300):
      print("Error: Resolution is too small. A minimum resolution of 300x300 is needed.")
      video.release()
      return False

    #Perform histogram equalization to increase visibility ----------------------------
    #https://www.geeksforgeeks.org/saving-a-video-using-opencv/
    #We do not perform Histogram equalization anymore because AlphaPose also
    #did not use it during training
    # out_video_path = 'input/video' + long_nr + '_equal.mp4'
    # out_file = cv2.VideoWriter(out_video_path, cv2.VideoWriter_fourcc(*'mp4v'),
    #                         fps, (width, height))
    #Inspiration from: https://docs.opencv.org/3.4/dd/d43/tutorial_py_video_display.html
    # while True:
    #   ret, frame = video.read()
    #   if not ret:
    #     break
    #   grey = cv2.cvtColor(frame, cv2.COLOR_BGR2YCrCb)
    #   grey[:,:,0] = cv2.equalizeHist(grey[:,:,0])
    #   rgb_equal = cv2.cvtColor(grey, cv2.COLOR_YCrCb2BGR)
      #out_file.write(rgb_equal)

    # Release video capture and writer objects
    # video.release()
    # out_file.release()
    #-----------------------------------------------------------------------------------

    # Closes all the frames
    cv2.destroyAllWindows()
    return True

rvalue = video_preprocessing(video_path)
if not rvalue:
  print("Something went wrong, check the above error message before continuing.")
else:
  print("Everything looks good!")

Everything looks good!


# Pose Estimation is performed with AlphaPose

In [10]:
command = "python video_demo.py --video {} --outdir output --save_video --sp --vis_fast".format(video_path)
!{command}

Loading YOLO model..
Loading pose model from ./models/sppe/duc_se.pth
100% 62/62 [00:06<00:00, 10.00it/s]
===========================> Finish Model Running.


#Get Phase Estimation of each frame

In [11]:
cd ../PhaseDetection

/content/TennisCoach/PhaseDetection


In [12]:
import VideoPhaseEstimator

The console stream is logged into /root/sg_logs/console.log


[2024-01-25 10:40:43] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it
[2024-01-25 10:40:44] WARNING - __init__.py - Failed to import pytorch_quantization
[2024-01-25 10:40:44] INFO - utils.py - NumExpr defaulting to 2 threads.
[2024-01-25 10:40:54] WARNING - calibrator.py - Failed to import pytorch_quantization
[2024-01-25 10:40:54] WARNING - export.py - Failed to import pytorch_quantization
[2024-01-25 10:40:54] WARNING - selective_quantization_utils.py - Failed to import pytorch_quantization
[2024-01-25 10:40:54] WARNING - env_sanity_check.py - Failed to verify installed packages: boto3 required but not found
[2024-01-25 10:40:54] WARNING - env_sanity_check.py - Failed to verify installed packages: deprecated required but not found
[2024-01-25 10:40:54] WARNING - env_sanity_check.py - Failed to verify installed packages: coverage required but not found
[2024-01-25 10:40:55] WARNING - env_sanity_check.py - 

In [13]:
path = "/content/TennisCoach/AlphaPose/" + video_path
video_phase_estimator = VideoPhaseEstimator.VideoPhaseEstimator()
frame_phase = video_phase_estimator.estimate_phases(path)

#store frame number for each phase in list for keypoint analysis
start_nr = [idx for idx, v in enumerate(frame_phase) if v == 'start']
trophy_nr = [idx for idx, v in enumerate(frame_phase) if v == 'loading']
acc_nr = [idx for idx, v in enumerate(frame_phase) if v == 'acceleration']
contact_nr = [idx for idx, v in enumerate(frame_phase) if v == 'contact']
finish_nr = [idx for idx, v in enumerate(frame_phase) if v == 'finish']

#To have time intervals for final annotated video
s_start = min(start_nr) + (max(start_nr)-min(start_nr))/2 if min(start_nr) + (max(start_nr)-min(start_nr))/2 >= 0 else 0
s_trophy = max(start_nr)
s_acc = max(trophy_nr)
s_contact = max(acc_nr)
s_finish = max(contact_nr)
e_finish = max(finish_nr)

[2024-01-25 10:40:57] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.
Downloading: "https://sghub.deci.ai/models/yolo_nas_l_coco.pth" to /root/.cache/torch/hub/checkpoints/yolo_nas_l_coco.pth
100%|██████████| 256M/256M [00:01<00:00, 180MB/s]
[2024-01-25 10:41:00] INFO - checkpoint_utils.py - Successfully loaded pretrained weights for architecture yolo_nas_l
[2024-01-25 10:41:01] INFO - checkpoint_utils.py - License Notification: YOLO-NAS-POSE pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS-POSE.md
By downloading the pre-trained weight files you agree to comply with these terms.
Downloading: "https://sghub.deci.ai/mode

#Find mistakes in each extracted phase:

In [14]:
cd /content/TennisCoach/AlphaPose

In [15]:
import json
import math
import numpy as np

#Have an array with error codes we can then use in the visualization to extract
#bounding boxes around needed areas
mistakes = []
# Mistakes in Start Phase:
#  0 - Left foot not in front of right foot
#  1 - Feet are too close together
#  2 - Feet are too far apart
#  3 - Let your left arm dangle
#  4 - Let your right arm dangle
# Mistakes in Trophy Phase:
#  5 - Right arm should be in a 90° angle - under 80°
#  6 - Right arm should be in a 90° angle - over 100°
#  7 - Left arm should be already above your head
#  8 - Left arm should be more stretched
#  9 - Bow your shoulder, hips and feet more
# 10 - Bow your left and right body part equally
# Mistakes in Acceleration Phase:
# 11 - bow your shoulder, hips and feet more
# 12 - bow your left and right body part equally
# Mistakes in Contact Point Phase:
# 13 - straight arm - angle between shoulder - elbow - wrist ~ 180°
# 14 - stretch left leg more
# 15 - stretch right leg more
# Mistakes in Finish Phase:
# 16 - bend knees more to protect knees (compared to contact point phase)

def checkPosition():
  with open('output/alphapose-results.json') as json_result:
    data = json.load(json_result)  #list of dictionaries

  #1. remove additional persons --------------------------------------------
  #indices are added to list 'idx_to_remove'
  idx_to_remove = []
  for idx, dic in enumerate(data):
    if idx > 0 and dic['image_id'] == data[idx-1]['image_id']:
      for idx2 in range(idx-1, -1, -1):
        if dic['image_id'] != data[idx2]['image_id']:
          break
        if idx2 in idx_to_remove:
          continue
        height_1 = max(data[idx2]['keypoints'][46] - data[idx2]['keypoints'][1], data[idx2]['keypoints'][49] - data[idx2]['keypoints'][1])
        height_2 = max(dic['keypoints'][46] - dic['keypoints'][1], dic['keypoints'][49] - dic['keypoints'][1])
        if(height_2 > height_1):
          idx_to_remove.append(idx2)
        else:
          idx_to_remove.append(idx)
        break

  idx_to_remove = list(dict.fromkeys(idx_to_remove)) # remove duplicates
  idx_to_remove.sort(reverse=True) #sorts indices in descending order - so we don't have an indexing error
  #removes additional persons
  for idx in idx_to_remove:
    data.pop(idx)

  with open('output/alphapose-results.json', "w") as json_file:
    json.dump(data, json_file)

  start_dicts, trophy_dicts, acc_dicts, contact_dicts, finish_dicts = [], [], [], [], []
  for idx, dic in enumerate(data):
    if idx in start_nr:
      start_dicts.append(dic)
    elif idx in trophy_nr:
      trophy_dicts.append(dic)
    elif idx in acc_nr:
      acc_dicts.append(dic)
    elif idx in contact_nr:
      contact_dicts.append(dic)
    elif idx in finish_nr:
      finish_dicts.append(dic)

  #-------------------------------------------------------------------------
  #2. check start phase ----------------------------------------------------
  # We concentrate on the analysis of videos taken from the right side and right-handed people
  mistakes.extend([0,1,2,3,4])
  for start_dict in start_dicts:
    keypoints_start = start_dict['keypoints']
    #keypoint order from https://github.com/MVIG-SJTU/AlphaPose/blob/master/docs/output.md
    nose = (keypoints_start[0], keypoints_start[1])
    left_eye = (keypoints_start[3], keypoints_start[4])
    right_eye = (keypoints_start[6], keypoints_start[7])
    left_ear = (keypoints_start[9], keypoints_start[10])
    right_ear = (keypoints_start[12], keypoints_start[13])
    left_shoulder = (keypoints_start[15], keypoints_start[16])
    right_shoulder = (keypoints_start[18], keypoints_start[19])
    left_elbow = (keypoints_start[21], keypoints_start[22])
    right_elbow = (keypoints_start[24], keypoints_start[25])
    left_wrist = (keypoints_start[27], keypoints_start[28])
    right_wrist = (keypoints_start[30], keypoints_start[31])
    left_hip = (keypoints_start[33], keypoints_start[34])
    right_hip = (keypoints_start[36], keypoints_start[37])
    left_knee = (keypoints_start[39], keypoints_start[40])
    right_knee = (keypoints_start[42], keypoints_start[43])
    left_ankle = (keypoints_start[45], keypoints_start[46])
    right_ankle = (keypoints_start[48], keypoints_start[49])

    #2.1. Left foot in front of right foot
    if(left_ankle[0] > right_ankle[0]):
      if 0 in mistakes:
        mistakes.remove(0)

    #2.2. Feet are approximately shoulder-width apart or slightly wider
    #Transform x-y coordinates to cm with resolution
    distance_feet_x = left_ankle[0] - right_ankle[0]
    distance_feet_y = left_ankle[1] - right_ankle[1]
    distance_feet = math.sqrt(distance_feet_x**2 + distance_feet_y**2)
    distance_shoulder_x = left_shoulder[0] - right_shoulder[0]
    distance_shoulder_y = left_shoulder[1] - right_shoulder[1]
    distance_shoulder = math.sqrt(distance_shoulder_x**2 + distance_shoulder_y**2)

    if distance_feet > distance_shoulder*1.1:
      if 1 in mistakes:
        mistakes.remove(1)
    if distance_feet < distance_shoulder*1.5:
      if 2 in mistakes:
       mistakes.remove(2)

    #Hands are pointing down - wrists under elbows - elbows under shoulder
    if not(left_wrist[1] < left_elbow[1] and left_elbow[1] < left_shoulder[1] and left_wrist[1] < left_hip[1]):
      if 3 in mistakes:
        mistakes.remove(3)
    if not(right_wrist[1] < right_elbow[1] and right_elbow[1] < right_shoulder[1] and right_wrist[1] < right_hip[1]):
      if 4 in mistakes:
        mistakes.remove(4)

  #-------------------------------------------------------------------------
  #3. check trophy phase ---------------------------------------------------
  mistakes.extend([5,6,7,8,9,10])
  for trophy_dict in trophy_dicts:
    keypoints_trophy = trophy_dict['keypoints']
    #keypoint order from https://github.com/MVIG-SJTU/AlphaPose/blob/master/docs/output.md
    nose = (keypoints_trophy[0], keypoints_trophy[1])
    left_eye = (keypoints_trophy[3], keypoints_trophy[4])
    right_eye = (keypoints_trophy[6], keypoints_trophy[7])
    left_ear = (keypoints_trophy[9], keypoints_trophy[10])
    right_ear = (keypoints_trophy[12], keypoints_trophy[13])
    left_shoulder = (keypoints_trophy[15], keypoints_trophy[16])
    right_shoulder = (keypoints_trophy[18], keypoints_trophy[19])
    left_elbow = (keypoints_trophy[21], keypoints_trophy[22])
    right_elbow = (keypoints_trophy[24], keypoints_trophy[25])
    left_wrist = (keypoints_trophy[27], keypoints_trophy[28])
    right_wrist = (keypoints_trophy[30], keypoints_trophy[31])
    left_hip = (keypoints_trophy[33], keypoints_trophy[34])
    right_hip = (keypoints_trophy[36], keypoints_trophy[37])
    left_knee = (keypoints_trophy[39], keypoints_trophy[40])
    right_knee = (keypoints_trophy[42], keypoints_trophy[43])
    left_ankle = (keypoints_trophy[45], keypoints_trophy[46])
    right_ankle = (keypoints_trophy[48], keypoints_trophy[49])

    # 2.1. 90° angle of right arm
    elbow_wrist = np.array([right_wrist[0] - right_elbow[0], right_wrist[1] - right_elbow[1]])
    elbow_shoulder = np.array([right_shoulder[0] - right_elbow[0], right_shoulder[1] - right_elbow[1]])
    # Formula from https://www.cuemath.com/geometry/angle-between-vectors/
    rad_angle = math.acos(np.dot(elbow_wrist,elbow_shoulder) / (np.linalg.norm(elbow_wrist) * np.linalg.norm(elbow_shoulder)))
    deg_angle = rad_angle * ( 180.0 / math.pi)
    if deg_angle < 80:
      if 5 in mistakes:
        mistakes.remove(5)
    if deg_angle < 100:
      if 6 in mistakes:
        mistakes.remove(6)

    # 2.2. left hand up
    # 2.2.1. left wrist over nose
    if left_wrist[1] < nose[1]:
      if 7 in mistakes:
        mistakes.remove(7)

    # 2.2.2. angle of left arm between 90 and 180°
    l_elbow_wrist = np.array([left_wrist[0] - left_elbow[0], left_wrist[1] - left_elbow[1]])
    l_elbow_shoulder = np.array([left_shoulder[0] - left_elbow[0], left_shoulder[1] - left_elbow[1]])
    rad_angle = math.acos(np.dot(l_elbow_wrist,l_elbow_shoulder) / (np.linalg.norm(l_elbow_wrist) * np.linalg.norm(l_elbow_shoulder)))
    deg_angle = rad_angle * ( 180.0 / math.pi)
    if deg_angle >= 90: #I guess stetching an arm over 180° is not possible!
      if 8 in mistakes:
        mistakes.remove(8)

    # 2.3. bow of shoulder, hips and feet
    # check if angle is smaller than 180°
    r_hip_shoulder = np.array([right_shoulder[0] - right_hip[0], right_shoulder[1] - right_hip[1]])
    r_hip_ankle = np.array([right_ankle[0] - right_hip[0], right_ankle[1] - right_hip[1]])
    # Formula from https://www.cuemath.com/geometry/angle-between-vectors/
    r_rad_angle = math.acos(np.dot(r_hip_shoulder,r_hip_ankle) / (np.linalg.norm(r_hip_shoulder) * np.linalg.norm(r_hip_ankle)))
    r_deg_angle = r_rad_angle * (180.0 / math.pi)
    if not (r_deg_angle > 180 and right_shoulder[0] > right_hip[0] and right_ankle[0] > right_hip[0]):
      if 9 in mistakes:
        mistakes.remove(9)

    l_hip_shoulder = np.array([left_shoulder[0] - left_hip[0], left_shoulder[1] - left_hip[1]])
    l_hip_ankle = np.array([left_ankle[0] - left_hip[0], left_ankle[1] - left_hip[1]])
    l_rad_angle = math.acos(np.dot(l_hip_shoulder,l_hip_ankle) / (np.linalg.norm(l_hip_shoulder) * np.linalg.norm(l_hip_ankle)))
    l_deg_angle = l_rad_angle * (180.0 / math.pi)
    lower_bound = r_deg_angle * 0.9
    upper_bound = r_deg_angle * 1.1
    if l_deg_angle > lower_bound and l_deg_angle < upper_bound:
      if 10 in mistakes:
        mistakes.remove(10)

  #-------------------------------------------------------------------------
  #3. check acceleration phase ---------------------------------------------
  mistakes.extend([11,12])
  for acc_dict in acc_dicts:
    keypoints_acc = acc_dict['keypoints']
    #keypoint order from https://github.com/MVIG-SJTU/AlphaPose/blob/master/docs/output.md
    nose = (keypoints_acc[0], keypoints_acc[1])
    left_eye = (keypoints_acc[3], keypoints_acc[4])
    right_eye = (keypoints_acc[6], keypoints_acc[7])
    left_ear = (keypoints_acc[9], keypoints_acc[10])
    right_ear = (keypoints_acc[12], keypoints_acc[13])
    left_shoulder = (keypoints_acc[15], keypoints_acc[16])
    right_shoulder = (keypoints_acc[18], keypoints_acc[19])
    left_elbow = (keypoints_acc[21], keypoints_acc[22])
    right_elbow = (keypoints_acc[24], keypoints_acc[25])
    left_wrist = (keypoints_acc[27], keypoints_acc[28])
    right_wrist = (keypoints_acc[30], keypoints_acc[31])
    left_hip = (keypoints_acc[33], keypoints_acc[34])
    right_hip = (keypoints_acc[36], keypoints_acc[37])
    left_knee = (keypoints_acc[39], keypoints_acc[40])
    right_knee = (keypoints_acc[42], keypoints_acc[43])
    left_ankle = (keypoints_acc[45], keypoints_acc[46])
    right_ankle = (keypoints_acc[48], keypoints_acc[49])

    # 3.2. bow of shoulder, hips and feet
    # check if angle is smaller than 180°
    r_hip_shoulder = np.array([right_shoulder[0] - right_hip[0], right_shoulder[1] - right_hip[1]])
    r_hip_ankle = np.array([right_ankle[0] - right_hip[0], right_ankle[1] - right_hip[1]])
    r_rad_angle = math.acos(np.dot(r_hip_shoulder,r_hip_ankle) / (np.linalg.norm(r_hip_shoulder) * np.linalg.norm(r_hip_ankle)))
    r_deg_angle = r_rad_angle * (180.0 / math.pi)
    if r_deg_angle < 180 and right_shoulder[0] < right_hip[0] and right_ankle[0] < right_hip[0]:
      if 11 in mistakes:
        mistakes.remove(11)

    l_hip_shoulder = np.array([left_shoulder[0] - left_hip[0], left_shoulder[1] - left_hip[1]])
    l_hip_ankle = np.array([left_ankle[0] - left_hip[0], left_ankle[1] - left_hip[1]])
    l_rad_angle = math.acos(np.dot(l_hip_shoulder,l_hip_ankle) / (np.linalg.norm(l_hip_shoulder) * np.linalg.norm(l_hip_ankle)))
    l_deg_angle = l_rad_angle * (180.0 / math.pi)
    lower_bound = r_deg_angle * 0.9
    upper_bound = r_deg_angle * 1.1
    if l_deg_angle > lower_bound and l_deg_angle < upper_bound:
      if 12 in mistakes:
        mistakes.remove(12)

  #-------------------------------------------------------------------------
  # 4. check contact point phase -------------------------------------------
  mistakes.extend([13,14,15])
  for contact_dict in contact_dicts:
    keypoints_contact = contact_dict['keypoints']
    #keypoint order from https://github.com/MVIG-SJTU/AlphaPose/blob/master/docs/output.md
    nose = (keypoints_contact[0], keypoints_contact[1])
    left_eye = (keypoints_contact[3], keypoints_contact[4])
    right_eye = (keypoints_contact[6], keypoints_contact[7])
    left_ear = (keypoints_contact[9], keypoints_contact[10])
    right_ear = (keypoints_contact[12], keypoints_contact[13])
    left_shoulder = (keypoints_contact[15], keypoints_contact[16])
    right_shoulder = (keypoints_contact[18], keypoints_contact[19])
    left_elbow = (keypoints_contact[21], keypoints_contact[22])
    right_elbow = (keypoints_contact[24], keypoints_contact[25])
    left_wrist = (keypoints_contact[27], keypoints_contact[28])
    right_wrist = (keypoints_contact[30], keypoints_contact[31])
    left_hip = (keypoints_contact[33], keypoints_contact[34])
    right_hip = (keypoints_contact[36], keypoints_contact[37])
    left_knee = (keypoints_contact[39], keypoints_contact[40])
    right_knee = (keypoints_contact[42], keypoints_contact[43])
    left_ankle = (keypoints_contact[45], keypoints_contact[46])
    right_ankle = (keypoints_contact[48], keypoints_contact[49])

    # 4.1. straight arm - angle between shoulder - elbow - wrist = 180°
    elbow_wrist = np.array([right_wrist[0] - right_elbow[0], right_wrist[1] - right_elbow[1]])
    elbow_shoulder = np.array([right_shoulder[0] - right_elbow[0], right_shoulder[1] - right_elbow[1]])
    rad_angle = math.acos(np.dot(elbow_wrist,elbow_shoulder) / (np.linalg.norm(elbow_wrist) * np.linalg.norm(elbow_shoulder)))
    deg_angle = rad_angle * (180.0 / math.pi)
    if deg_angle > 170:
      if 13 in mistakes:
        mistakes.remove(13)

    # 4.2. hitting point at maximum height (not sure, how to do that)  (TODO - ask Sebastian)
    #check if legs are also stretched
    #left leg
    left_knee_ankle = np.array([left_knee[0] - left_ankle[0], left_knee[1] - left_ankle[1]])
    left_knee_hip = np.array([left_knee[0] - left_hip[0], left_knee[1] - left_hip[1]])
    rad_angle = math.acos(np.dot(left_knee_ankle,left_knee_hip) / (np.linalg.norm(left_knee_ankle) * np.linalg.norm(left_knee_hip)))
    deg_angle = rad_angle * (180.0 / math.pi)
    if deg_angle > 165:
      if 14 in mistakes:
        mistakes.remove(14)

    #right leg
    right_knee_ankle = np.array([right_knee[0] - right_ankle[0], right_knee[1] - right_ankle[1]])
    right_knee_hip = np.array([right_knee[0] - right_hip[0], right_knee[1] - right_hip[1]])
    rad_angle = math.acos(np.dot(right_knee_ankle,right_knee_hip) / (np.linalg.norm(right_knee_ankle) * np.linalg.norm(right_knee_hip)))
    deg_angle = rad_angle * (180.0 / math.pi)
    if deg_angle > 165:
      if 15 in mistakes:
        mistakes.remove(15)

  #-------------------------------------------------------------------------
  # 5. check finish phase --------------------------------------------------

  #angles from contact point phase to compare with angle from finish phase
  prev_l_knee_ankle = np.array([left_ankle[0] - left_knee[0], left_ankle[1] - left_knee[1]])
  prev_l_knee_hip = np.array([left_knee[0] - left_hip[0], left_knee[1] - left_hip[1]])
  prev_l_rad_angle = math.acos(np.dot(prev_l_knee_ankle,prev_l_knee_hip) / (np.linalg.norm(prev_l_knee_ankle) * np.linalg.norm(prev_l_knee_hip)))
  prev_l_deg_angle = prev_l_rad_angle * (180.0 / math.pi)

  prev_r_knee_ankle = np.array([right_ankle[0] - right_knee[0], right_ankle[1] - right_knee[1]])
  prev_r_knee_hip = np.array([right_knee[0] - right_hip[0], right_knee[1] - right_hip[1]])
  prev_r_rad_angle = math.acos(np.dot(prev_r_knee_ankle,prev_r_knee_hip) / (np.linalg.norm(prev_r_knee_ankle) * np.linalg.norm(prev_r_knee_hip)))
  prev_r_deg_angle = prev_r_rad_angle * (180.0 / math.pi)

  mistakes.append(16)
  for finish_dict in finish_dicts:
    keypoints_finish = finish_dict['keypoints']
    #keypoint order from https://github.com/MVIG-SJTU/AlphaPose/blob/master/docs/output.md
    nose = (keypoints_finish[0], keypoints_finish[1])
    left_eye = (keypoints_finish[3], keypoints_finish[4])
    right_eye = (keypoints_finish[6], keypoints_finish[7])
    left_ear = (keypoints_finish[9], keypoints_finish[10])
    right_ear = (keypoints_finish[12], keypoints_finish[13])
    left_shoulder = (keypoints_finish[15], keypoints_finish[16])
    right_shoulder = (keypoints_finish[18], keypoints_finish[19])
    left_elbow = (keypoints_finish[21], keypoints_finish[22])
    right_elbow = (keypoints_finish[24], keypoints_finish[25])
    left_wrist = (keypoints_finish[27], keypoints_finish[28])
    right_wrist = (keypoints_finish[30], keypoints_finish[31])
    left_hip = (keypoints_finish[33], keypoints_finish[34])
    right_hip = (keypoints_finish[36], keypoints_finish[37])
    left_knee = (keypoints_finish[39], keypoints_finish[40])
    right_knee = (keypoints_finish[42], keypoints_finish[43])
    left_ankle = (keypoints_finish[45], keypoints_finish[46])
    right_ankle = (keypoints_finish[48], keypoints_finish[49])

    # 5.1. bend knees
    left_knee_ankle = np.array([left_ankle[0] - left_knee[0], left_ankle[1] - left_knee[1]])
    left_knee_hip = np.array([left_knee[0] - left_hip[0], left_knee[1] - left_hip[1]])
    l_rad_angle = math.acos(np.dot(left_knee_ankle,left_knee_hip) / (np.linalg.norm(left_knee_ankle) * np.linalg.norm(left_knee_hip)))
    l_deg_angle = l_rad_angle * (180.0 / math.pi)

    right_knee_ankle = np.array([right_ankle[0] - right_knee[0], right_ankle[1] - right_knee[1]])
    right_knee_hip = np.array([right_knee[0] - right_hip[0], right_knee[1] - right_hip[1]])
    r_rad_angle = math.acos(np.dot(right_knee_ankle,right_knee_hip) / (np.linalg.norm(right_knee_ankle) * np.linalg.norm(right_knee_hip)))
    r_deg_angle = r_rad_angle * (180.0 / math.pi)

    # compare bending to previous ones
    left_bound = prev_l_deg_angle * 0.85
    right_bound = prev_r_deg_angle * 0.85
    if left_bound < l_deg_angle and right_bound < prev_r_deg_angle:
      if 16 in mistakes:
        mistakes.remove(16)

checkPosition()


#Generate final output video:

In [16]:
import cv2

long_nr = str('{:03d}'.format(video_nr))
video_path = 'input/video' + long_nr + '.mp4'
video = cv2.VideoCapture(video_path)

if video.isOpened():
  nr_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
  fps = video.get(cv2.CAP_PROP_FPS)
  width = int(video.get(3))
  height = int(video.get(4))
  resolution = (width, height)
  print(f"Resolution: {width}x{height}")

  out_video_path = 'output/annotated_video' + long_nr + '.mp4'
  out_file = cv2.VideoWriter(out_video_path, cv2.VideoWriter_fourcc(*'mp4v'), 7, (width, height))

  with open('output/alphapose-results.json') as json_result:
      data = json.load(json_result)  #list of dictionaries

  counter = 0
  thickness = 3
  radius_0 = None
  radius_1_2 = None
  radius_7 = None
  while True:
    success, frame = video.read()
    if not success:
      break
    keypoints = data[counter]['keypoints']
    if counter >= s_start and counter < s_trophy:
      #check error codes 0,1,2,3,4
      text = []
      if (1 in mistakes) or (2 in mistakes):
        left_ankle = (keypoints[45], keypoints[46])
        right_ankle = (keypoints[48], keypoints[49])
        center = (int(min(right_ankle[0], left_ankle[0]) + abs(right_ankle[0] - left_ankle[0])/2),
                  int(min(right_ankle[1], left_ankle[1]) + abs(right_ankle[1] - left_ankle[1])/2))
        if radius_1_2 == None:
          radius_1_2 = (int(abs(right_ankle[0] - left_ankle[0]) * 1.1), int((keypoints[46] - keypoints[40])/2.0))
        angle = 0
        color = (84, 139, 84) # green in BGR
        frame = cv2.ellipse(frame, center, radius_1_2, 0, 0, 360, color, thickness)
        # add text
        text.append((color, "Your feet should be slightly wider"))
        text.append((color, "than shoulder-width apart."))
      else:
        text.append(((0,0,0),"Your feet have an ideal distance."))

      if 0 in mistakes:
        left_ankle = (int(keypoints[45]), int(keypoints[46]))
        if radius_0 == None:
          radius_0 = int((keypoints[46] - keypoints[40])/2.0) #(left_ankle.y - left_knee.y) / 2
        color = (0, 165, 255) # orange in BGR
        frame = cv2.circle(frame, left_ankle, radius_0, color, thickness)
        text.append((color,"Your left foot should be in front"))
        text.append((color,"of the right one."))
      else:
        text.append(((0,0,0), "Your left foot is positioned correctly."))

      if 3 in mistakes:
        color = (57, 79, 205)
        left_shoulder = (int(keypoints[15]), int(keypoints[16]))
        left_elbow = (int(keypoints[21]), int(keypoints[22]))
        left_wrist = (int(keypoints[27]), int(keypoints[28]))
        frame = cv2.line(frame, left_shoulder, left_elbow, color, thickness)
        frame = cv2.line(frame, left_elbow, left_wrist, color, thickness)
        text.append((color,"Let your left arm dangle!"))
      else:
        text.append(((0,0,1), "The position of your left arm"))
        text.append(((0,0,1), "is perfect."))

      if 4 in mistakes:
        color = (237, 149, 100)
        right_shoulder = (int(keypoints[18]), int(keypoints[19]))
        right_elbow = (int(keypoints[24]), int(keypoints[25]))
        right_wrist = (int(keypoints[30]), int(keypoints[31]))
        frame = cv2.line(frame, right_shoulder, right_elbow, color, thickness)
        frame = cv2.line(frame, right_elbow, right_wrist, color, thickness)
        text.append((color, "Let your right arm dangle!"))
      else:
        text.append(((0,0,2), "The position of your right arm"))
        text.append(((0,0,2), "is perfect."))

      frame_cpy = frame.copy()
      vspace = height * 0.03
      hspace = width * 0.04
      frame = cv2.rectangle(frame, (int(width*2/3), int(height*2/3)), (width, height), (222, 222, 222), cv2.FILLED)
      vadd = 0
      for i, (c,t) in enumerate(text):
        if(i> 0 and c != (0,0,0) and c == text[i-1][0]):
          vadd += vspace
        else:
          vadd += vspace*2
          if(c == (0,0,0) or  c == (0,0,1) or c == (0,0,2)):
            frame = cv2.circle(frame, (int(width*2/3+(hspace/2)), int(height*2/3+vadd-vspace/4)), int(vspace/2), c, 1)
          else:
            frame = cv2.circle(frame, (int(width*2/3+(hspace/2)), int(height*2/3+vadd-vspace/4)), int(vspace/2), c, cv2.FILLED)
        frame = cv2.putText(frame, t, (int(width*2/3+hspace), int(height*2/3+vadd)), cv2.FONT_HERSHEY_DUPLEX, 0.6, (22, 22, 22), 1)
      alpha = 0.7
      frame=cv2.addWeighted(frame, alpha, frame_cpy,1-alpha, gamma=0)
    #---------------------------------------------------------------------------
    elif counter >= s_trophy and counter < s_acc:
      #check error codes 5,6,7,8,9,10
      text = []
      if 5 in mistakes or 6 in mistakes:
        color = (237, 149, 100)
        right_shoulder = (int(keypoints[18]), int(keypoints[19]))
        right_elbow = (int(keypoints[24]), int(keypoints[25]))
        right_wrist = (int(keypoints[30]), int(keypoints[31]))
        frame = cv2.line(frame, right_shoulder, right_elbow, color, thickness)
        frame = cv2.line(frame, right_elbow, right_wrist, color, thickness)
        text.append((color, "Your right arm should be in"))
        text.append((color, "a 90 degree angle."))
      else:
        text.append(((0,0,0), "Your right arm has an ideal angle."))

      if 7 in mistakes:
        left_wrist = (int(keypoints[27]), int(keypoints[28]))
        if radius_7 == None:
          radius_7 = int((keypoints[46] - keypoints[40])/2.0) #(left_ankle.y - left_knee.y) / 2
        color = (0, 165, 255) # orange in BGR
        frame = cv2.circle(frame, left_wrist, radius_7, color, thickness)
        text.append((color, "Your left arm should be up."))
      else:
        text.append(((0,0,0), "Your left arm has an ideal position."))

      if 8 in mistakes:
        color = (57, 79, 205)
        left_shoulder = (int(keypoints[15]), int(keypoints[16]))
        left_elbow = (int(keypoints[21]), int(keypoints[22]))
        left_wrist = (int(keypoints[27]), int(keypoints[28]))
        frame = cv2.line(frame, left_shoulder, left_elbow, color, thickness)
        frame = cv2.line(frame, left_elbow, left_wrist, color, thickness)
        text.append((color, "Stretch your left arm more."))
      else:
        text.append(((0,0,0), "Your left arm is correctly stretched."))

      if 9 in mistakes: # or 10 in mistakes:
        pt_x = []
        pt_y = []
        right_shoulder = (keypoints[18], keypoints[19])
        right_hip = (keypoints[36], keypoints[37])
        right_ankle = (keypoints[48], keypoints[49])
        #Bézier curves
        for t in np.linspace(0, 1, 100):
          x = (1 - t)**2 * right_shoulder[0] + 2 * (1 - t) * t * right_hip[0] + t**2 * right_ankle[0]
          y = (1 - t)**2 * right_shoulder[1] + 2 * (1 - t) * t * right_hip[1] + t**2 * right_ankle[1]
          pt_x.append(int(x))
          pt_y.append(int(y))
        pt = np.concatenate((np.array(pt_x).reshape(-1, 1), np.array(pt_y).reshape(-1, 1)), axis=1)
        #https://www.geeksforgeeks.org/python-opencv-cv2-polylines-method/
        color = (205, 82, 180) # green in BGR
        frame = cv2.polylines(frame, [pt.reshape((-1, 1, 2))], False, color, thickness)
        text.append((color, "Tilt your body more backward."))
      else:
        text.append(((0,0,0), "You have an ideal posture."))

      frame_cpy = frame.copy()
      vspace = height * 0.03
      hspace = width * 0.04
      frame = cv2.rectangle(frame, (int(width*2/3), int(height*2/3)), (width, height), (222, 222, 222), cv2.FILLED)
      vadd = 0
      for i, (c,t) in enumerate(text):
        if(i> 0 and c != (0,0,0) and c == text[i-1][0]):
          vadd += vspace
        else:
          vadd += vspace*2
          if(c == (0,0,0)):
            frame = cv2.circle(frame, (int(width*2/3+(hspace/2)), int(height*2/3+vadd-vspace/4)), int(vspace/2), c, 1)
          else:
            frame = cv2.circle(frame, (int(width*2/3+(hspace/2)), int(height*2/3+vadd-vspace/4)), int(vspace/2), c, cv2.FILLED)
        frame = cv2.putText(frame, t, (int(width*2/3+hspace), int(height*2/3+vadd)), cv2.FONT_HERSHEY_DUPLEX, 0.6, (22, 22, 22), 1)
      alpha = 0.7
      frame=cv2.addWeighted(frame, alpha, frame_cpy,1-alpha, gamma=0)

    elif counter >= s_acc and counter < s_contact:
      #check error codes 11, 12
      text = []
      if 11 in mistakes: # or 12 not in mistakes:
        pt_x = []
        pt_y = []
        right_shoulder = (keypoints[18], keypoints[19])
        right_hip = (keypoints[36], keypoints[37])
        right_ankle = (keypoints[48], keypoints[49])
        #Bézier curves
        for t in np.linspace(0, 1, 100):
          x = (1 - t)**2 * right_shoulder[0] + 2 * (1 - t) * t * right_hip[0] + t**2 * right_ankle[0]
          y = (1 - t)**2 * right_shoulder[1] + 2 * (1 - t) * t * right_hip[1] + t**2 * right_ankle[1]
          pt_x.append(int(x))
          pt_y.append(int(y))
        pt = np.concatenate((np.array(pt_x).reshape(-1, 1), np.array(pt_y).reshape(-1, 1)), axis=1)
        #https://www.geeksforgeeks.org/python-opencv-cv2-polylines-method/
        color = (205, 82, 180) # green in BGR
        frame = cv2.polylines(frame, [pt.reshape((-1, 1, 2))], False, color, thickness)
        text.append((color, "Tilt your body more backward."))
      else:
        text.append(((0,0,0), "You have an ideal posture."))

      frame_cpy = frame.copy()
      vspace = height * 0.03
      hspace = width * 0.04
      frame = cv2.rectangle(frame, (int(width*2/3), int(height*2/3)), (width, height), (222, 222, 222), cv2.FILLED)
      vadd = 0
      for i, (c,t) in enumerate(text):
        if(i> 0 and c != (0,0,0) and c == text[i-1][0]):
          vadd += vspace
        else:
          vadd += vspace*2
          if(c == (0,0,0)):
            frame = cv2.circle(frame, (int(width*2/3+(hspace/2)), int(height*2/3+vadd-vspace/4)), int(vspace/2), c, 1)
          else:
            frame = cv2.circle(frame, (int(width*2/3+(hspace/2)), int(height*2/3+vadd-vspace/4)), int(vspace/2), c, cv2.FILLED)
        frame = cv2.putText(frame, t, (int(width*2/3+hspace), int(height*2/3+vadd)), cv2.FONT_HERSHEY_DUPLEX, 0.6, (22, 22, 22), 1)
      alpha = 0.7
      frame=cv2.addWeighted(frame, alpha, frame_cpy,1-alpha, gamma=0)

    elif counter >= s_contact and counter < s_finish:
      #check error codes 13, 14, 15
      text = []
      if 13 in mistakes:
        color = (205, 82, 180)
        right_shoulder = (int(keypoints[18]), int(keypoints[19]))
        right_elbow = (int(keypoints[24]), int(keypoints[25]))
        right_wrist = (int(keypoints[30]), int(keypoints[31]))
        frame = cv2.line(frame, right_shoulder, right_elbow, color, thickness)
        frame = cv2.line(frame, right_elbow, right_wrist, color, thickness)
        text.append((color, "Keep your right arm straight!"))
      else:
        text.append(((0,0,1), "You kept your right arm"))
        text.append(((0,0,1), "perfectly straight."))

      if 14 in mistakes:
        color = (57, 79, 205)
        left_hip = (int(keypoints[33]), int(keypoints[34]))
        left_knee = (int(keypoints[39]), int(keypoints[40]))
        left_ankle = (int(keypoints[45]), int(keypoints[46]))
        frame = cv2.line(frame, left_hip, left_knee, color, thickness)
        frame = cv2.line(frame, left_knee, left_ankle, color, thickness)
        text.append((color, "Stretch your left leg more!"))
      else:
        text.append(((0,0,2), "You kept your left leg"))
        text.append(((0,0,2), "perfectly straight."))

      if 15 in mistakes:
        color = (237, 149, 100)
        right_hip = (int(keypoints[36]), int(keypoints[37]))
        right_knee = (int(keypoints[42]), int(keypoints[43]))
        right_ankle = (int(keypoints[48]), int(keypoints[49]))
        frame = cv2.line(frame, right_hip, right_knee, color, thickness)
        frame = cv2.line(frame, right_knee, right_ankle, color, thickness)
        text.append((color, "Stretch your right leg more!"))
      else:
        text.append(((0,0,3), "You kept your right leg"))
        text.append(((0,0,3), "perfectly straight."))

      frame_cpy = frame.copy()
      vspace = height * 0.03
      hspace = width * 0.04
      frame = cv2.rectangle(frame, (int(width*2/3), int(height*2/3)), (width, height), (222, 222, 222), cv2.FILLED)
      vadd = 0
      for i, (c,t) in enumerate(text):
        if(i> 0 and c != (0,0,0) and c == text[i-1][0]):
          vadd += vspace
        else:
          vadd += vspace*2
          if(c == (0,0,0) or c == (0,0,1) or c == (0,0,2) or c == (0,0,3)):
            frame = cv2.circle(frame, (int(width*2/3+(hspace/2)), int(height*2/3+vadd-vspace/4)), int(vspace/2), c, 1)
          else:
            frame = cv2.circle(frame, (int(width*2/3+(hspace/2)), int(height*2/3+vadd-vspace/4)), int(vspace/2), c, cv2.FILLED)
        frame = cv2.putText(frame, t, (int(width*2/3+hspace), int(height*2/3+vadd)), cv2.FONT_HERSHEY_DUPLEX, 0.6, (22, 22, 22), 1)
      alpha = 0.7
      frame=cv2.addWeighted(frame, alpha, frame_cpy,1-alpha, gamma=0)

    elif counter >= s_finish and counter < e_finish:
      #check error codes 18
      text = []
      if 16 in mistakes:
        left_hip = (int(keypoints[33]), int(keypoints[34]))
        left_knee = (int(keypoints[39]), int(keypoints[40]))
        left_ankle = (int(keypoints[45]), int(keypoints[46]))
        right_hip = (int(keypoints[36]), int(keypoints[37]))
        right_knee = (int(keypoints[42]), int(keypoints[43]))
        right_ankle = (int(keypoints[48]), int(keypoints[49]))
        color = (57, 79, 205)
        frame = cv2.line(frame, left_hip, left_knee, color, thickness)
        frame = cv2.line(frame, left_knee, left_ankle, color, thickness)
        color = (237, 149, 100)
        frame = cv2.line(frame, right_hip, right_knee, color, thickness)
        frame = cv2.line(frame, right_knee, right_ankle, color, thickness)
        text.append((color, "Bend your knees more after"))
        text.append((color, "jumping to protect your knees."))
      else:
        text.append(((0,0,0), "Nice jump!"))

      frame_cpy = frame.copy()
      vspace = height * 0.03
      hspace = width * 0.04
      frame = cv2.rectangle(frame, (int(width*2/3), int(height*2/3)), (width, height), (222, 222, 222), cv2.FILLED)
      vadd = 0
      for i, (c,t) in enumerate(text):
        if(i> 0 and c != (0,0,0) and c == text[i-1][0]):
          vadd += vspace
        else:
          vadd += vspace*2
          if(c == (0,0,0)):
            frame = cv2.circle(frame, (int(width*2/3+(hspace/2)), int(height*2/3+vadd-vspace/4)), int(vspace/2), c, 1)
          else:
            frame = cv2.circle(frame, (int(width*2/3+(hspace/2)), int(height*2/3+vadd-vspace/4)), int(vspace/2), c, cv2.FILLED)
        frame = cv2.putText(frame, t, (int(width*2/3+hspace), int(height*2/3+vadd)), cv2.FONT_HERSHEY_DUPLEX, 0.6, (22, 22, 22), 1)
      alpha = 0.7
      frame=cv2.addWeighted(frame, alpha, frame_cpy,1-alpha, gamma=0)

    out_file.write(frame)
    counter += 1

  # Release video capture and writer objects
  video.release()
  out_file.release()

# Closes all the frames
cv2.destroyAllWindows()

Could not recognize phases
Could not recognize phases
Could not recognize phases
Could not recognize phases
Could not recognize phases
Could not recognize phases
Could not recognize phases
No Tennis Racket detected in the given image
No Tennis Racket detected in the given image
No Tennis Racket detected in the given image
No Tennis Racket detected in the given image
/content/TennisCoach/AlphaPose
68
62
[5, 6, 7, 8, 9, 10]
161.04434933365255
175.78202194882817
158.6273649010227
177.2150737173869
170.24682114488922
169.85979599223512
179.08272883272085
160.3521760183981
171.86992195473303
174.7605019916091
174.7605219328832
177.015633061071
158.00893813338135
Resolution: 1280x720
